In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from dateutil import parser
from datetime import datetime

# Scrape Car24.bg

In [2]:
strings = ['abarth', 'acura', 'alpina', 'audi', 
       'aston-martin','austin','alfa-romeo', 'bentley',
       'bertone', 'bugatti'
       'bmw', 'buick', 'cadillac',
       'chevrolet','chrysler','citroen', 
       'corvette', 'cupra', 'dacia','daewoo',
       'daihatsu','dodge', 
       'ds', 'dr', 'eagle',
        'ferrari','fiat', 'ford', 'fso', 'gaz',
       'gmc', 'gonow', 'great-wall',
       'haval', 'honda',
        'hyundai','hummer', 'ifa', 'infiniti','innocenti', 
       'isuzu','iveco', 'jaguar', 'jeep','kia',
        'lada','laforza','lamborghini','lancia',
       'land-rover','lotus',
        'lexus', 'lincoln', 'mahindra', 'maybach'
        'maserati', 'mazda','mclaren', 'mg',
        'mercedes-benz', 'mini',
        'mitsubishi', 'moskvich','oldsmobile',
        'nissan', 'opel', 
        'peugeot', 'porsche',
        'pontiac',
        'renault', 'rolls-royce',
        'rover', 'saab', 
        'seat', 'skoda',
        'ssang-yong','ssangyong','subaru','suzuki',
        'smart','tata','tazzari',
        'tesla', 'toyota', 'uaz',
        'trabant','volga','volvo', 'vw']

In [3]:
models = []
prices = []
years = []
kilometers = []
transmissions = []
engine_types = []
horsepowers = []

In [4]:
def scrape_current_page_Car24(): 
    for result in results:
        try:
            model = result.find("img")["alt"].replace("/ - [1]","")
            model = model.replace(' - [1]','')
            models.append(model.strip())
        except:
            models.append(np.nan)

        try:
            price = result.find("div",class_="title").find("span").text
            prices.append(price)
        except:
            prices.append(np.nan)

        try:
            year = result.find("div", class_= "data").text.split(" • ")[0].strip('\n')
            years.append(year.strip())
        except:
            years.append(np.nan)

        try:
            km = result.find("div", class_= "data").text.split(" • ")[1]
            kilometers.append(km.strip())
        except:
            kilometers.append(np.nan)

        try:
            engine_type = result.find("text").find(class_ = "info").text.split(", ")[0].replace("\n","")
            engine_types.append(engine_type.strip())
        except:
            engine_types.append(np.nan)

        try:
            horsepower = result.find("text").find(class_ = "info").text.split(", ")[1].replace(" к.с","")
            horsepowers.append(horsepower.strip())
        except:
            horsepowers.append(np.nan)

In [5]:
for string in strings:
    url = 'https://www.car24.bg/obiavi/' + string + '/page-1'
    count = 1
    while True:
       # Get URL
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        li_text = soup.find('li').text
        if 'Всички' in li_text:
            break
            
        # Results
        results = soup.findAll("item")
        if len(results) == 0:
            break
        scrape_current_page_Car24()
        count = count + 1
        next_url = 'https://www.car24.bg/obiavi/' + string + '/page-' + str(count)
        url = next_url

In [6]:
# create dataframe
df_auto = pd.DataFrame({'Model' : models,'Price': prices,'Year': years, 
                               'Kilometers' : kilometers,'Engine_type': engine_types,
                                'Horsepower': horsepowers})

In [7]:
df_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26070 entries, 0 to 26069
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Model        26070 non-null  object
 1   Price        26070 non-null  object
 2   Year         26070 non-null  object
 3   Kilometers   26070 non-null  object
 4   Engine_type  26070 non-null  object
 5   Horsepower   26070 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [8]:
df_auto.head(10)

,Model,Price,Year,Kilometers,Engine_type,Horsepower
0,Abarth 124,250 000 лв.,февруари 2020,11 111 км,Бензинов двигател,Автоматична ск.кутия
1,Abarth 595,45 800 лв.,юни 2019,54 350 км,Бензинов двигател,181
2,Abarth 595 TURISMO*500*PANORAMA,51 000 лв.,август 2021,8 200 км,Бензинов двигател,165
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40 900 лв.,декември 2017,48 000 км,Бензинов двигател,160
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32 500 лв.,август 2015,100 000 км,Бензинов двигател,278
5,Acura Mdx A-spec sh-awd 4x4,69 000 лв.,април 2019,62 000 км,Бензинов двигател,290
6,Acura Tsx,400 000 лв.,Април 1940,900 000 км,Дизелов двигател,Ръчна ск.кутия
7,Acura Tsx,400 000 лв.,Април 1940,900 000 км,Дизелов двигател,Ръчна ск.кутия
8,Acura Tsx,400 000 лв.,Юни 1938,300 000 км,Електрически двигател,Полуавтоматична ск.кутия
9,Acura Slx,900 000 лв.,Януари 2021,900 000 км,Бензинов двигател,Ръчна ск.кутия


# Data Cleaning

In [9]:
# convert currencies
d = {'EUR': 1.95, 'USD': 1.90, 'лв.': 1}
s1 = df_auto['Price'].replace(d, regex=True)
s2 = df_auto['Price'].str.extract('(\d+\ \d*)',expand = False)
s2 = s2.str.replace(' ','')
s2 = s2.astype(float)
df_auto['Price BGN'] = s1 * s2 
df_auto.head()

,Model,Price,Year,Kilometers,Engine_type,Horsepower,Price BGN
0,Abarth 124,250 000 лв.,февруари 2020,11 111 км,Бензинов двигател,Автоматична ск.кутия,250000.0
1,Abarth 595,45 800 лв.,юни 2019,54 350 км,Бензинов двигател,181,45800.0
2,Abarth 595 TURISMO*500*PANORAMA,51 000 лв.,август 2021,8 200 км,Бензинов двигател,165,51000.0
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40 900 лв.,декември 2017,48 000 км,Бензинов двигател,160,40900.0
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32 500 лв.,август 2015,100 000 км,Бензинов двигател,278,32500.0


In [10]:
del df_auto['Price']

In [11]:
df_auto = df_auto[['Model', 'Price BGN', 'Year', 'Kilometers', 'Engine_type', 'Horsepower']]
df_auto.head()

,Model,Price BGN,Year,Kilometers,Engine_type,Horsepower
0,Abarth 124,250000.0,февруари 2020,11 111 км,Бензинов двигател,Автоматична ск.кутия
1,Abarth 595,45800.0,юни 2019,54 350 км,Бензинов двигател,181
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,август 2021,8 200 км,Бензинов двигател,165
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,декември 2017,48 000 км,Бензинов двигател,160
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,август 2015,100 000 км,Бензинов двигател,278


In [12]:
df_auto.rename(columns={'Year':'Year Manifacture', 'Engine_type':'Engine Type'}, inplace=True)
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,февруари 2020,11 111 км,Бензинов двигател,Автоматична ск.кутия
1,Abarth 595,45800.0,юни 2019,54 350 км,Бензинов двигател,181
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,август 2021,8 200 км,Бензинов двигател,165
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,декември 2017,48 000 км,Бензинов двигател,160
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,август 2015,100 000 км,Бензинов двигател,278


In [13]:
# replace blank spaces with np.nan and clean Year Manifacture
df_auto = df_auto.replace(r'^\s*$', np.nan, regex=True)
df_auto = df_auto.dropna(subset = ["Year Manifacture"])
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,февруари 2020,11 111 км,Бензинов двигател,Автоматична ск.кутия
1,Abarth 595,45800.0,юни 2019,54 350 км,Бензинов двигател,181
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,август 2021,8 200 км,Бензинов двигател,165
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,декември 2017,48 000 км,Бензинов двигател,160
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,август 2015,100 000 км,Бензинов двигател,278


In [14]:
df_auto['Year Manifacture'].unique()

array(['февруари 2020', 'юни 2019', 'август 2021', 'декември 2017',
       'август 2015', 'април 2019', 'Април 1940', 'Юни 1938',
       'Януари 2021', 'април 2010', 'април 2015', 'януари 2013',
       'март 2019', 'юли 2011', 'септември 2013', 'септември 2012',
       'юли 2012', 'декември 2014', 'октомври 2012', 'март 2016',
       'януари 2016', 'януари 2017', 'януари 2008', 'февруари 2014',
       'февруари 2017', 'юли 2014', 'октомври 2019', 'февруари 2018',
       'януари 2009', 'юни 2014', 'ноември 2020', 'май 2018',
       'декември 2013', 'юли 2008', 'септември 2005', 'май 2011',
       'ноември 2019', 'януари 2011', 'октомври 2016', 'декември 2019',
       'март 2011', 'април 2016', 'август 2010', 'юни 2008',
       'октомври 2010', 'април 2012', 'декември 2022', 'март 2002',
       'септември 2017', 'януари 2006', 'август 2008', 'ноември 2011',
       'септември 2008', 'септември 2009', 'юни 2010', 'ноември 2018',
       'март 2009', 'декември 2008', 'октомври 2013', 'април 

In [15]:
# extract years from "Year Manifacture" 
df_auto['Year Manifacture'] = df_auto['Year Manifacture'].str.extract(r'(\d+)',expand = False)

In [16]:
# get only notna values
df_auto = df_auto[df_auto['Year Manifacture'].notna()]

In [17]:
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,2020,11 111 км,Бензинов двигател,Автоматична ск.кутия
1,Abarth 595,45800.0,2019,54 350 км,Бензинов двигател,181
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8 200 км,Бензинов двигател,165
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48 000 км,Бензинов двигател,160
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100 000 км,Бензинов двигател,278


In [18]:
df_auto["Year Manifacture_new"] = df_auto["Year Manifacture"].apply(lambda row: f'12/31/{row}')
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Year Manifacture_new
0,Abarth 124,250000.0,2020,11 111 км,Бензинов двигател,Автоматична ск.кутия,12/31/2020
1,Abarth 595,45800.0,2019,54 350 км,Бензинов двигател,181,12/31/2019
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8 200 км,Бензинов двигател,165,12/31/2021
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48 000 км,Бензинов двигател,160,12/31/2017
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100 000 км,Бензинов двигател,278,12/31/2015


In [19]:
# parce into DateTime
df_auto["Year Manifacture_new"] = [parser.parse(str(x)) for x in df_auto["Year Manifacture_new"]]
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Year Manifacture_new
0,Abarth 124,250000.0,2020,11 111 км,Бензинов двигател,Автоматична ск.кутия,2020-12-31
1,Abarth 595,45800.0,2019,54 350 км,Бензинов двигател,181,2019-12-31
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8 200 км,Бензинов двигател,165,2021-12-31
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48 000 км,Бензинов двигател,160,2017-12-31
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100 000 км,Бензинов двигател,278,2015-12-31


In [20]:
# extract "year" from Series
df_auto['Year Manifacture_t'] = df_auto['Year Manifacture_new'].dt.year
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Year Manifacture_new,Year Manifacture_t
0,Abarth 124,250000.0,2020,11 111 км,Бензинов двигател,Автоматична ск.кутия,2020-12-31,2020
1,Abarth 595,45800.0,2019,54 350 км,Бензинов двигател,181,2019-12-31,2019
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8 200 км,Бензинов двигател,165,2021-12-31,2021
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48 000 км,Бензинов двигател,160,2017-12-31,2017
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100 000 км,Бензинов двигател,278,2015-12-31,2015


In [21]:
del df_auto['Year Manifacture']
del df_auto['Year Manifacture_new']

In [22]:
df_auto.rename(columns={'Year Manifacture_t' : 'Year Manifacture'}, inplace=True)
df_auto.head()

,Model,Price BGN,Kilometers,Engine Type,Horsepower,Year Manifacture
0,Abarth 124,250000.0,11 111 км,Бензинов двигател,Автоматична ск.кутия,2020
1,Abarth 595,45800.0,54 350 км,Бензинов двигател,181,2019
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,8 200 км,Бензинов двигател,165,2021
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,48 000 км,Бензинов двигател,160,2017
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,100 000 км,Бензинов двигател,278,2015


In [23]:
df_auto = df_auto[['Model', 'Price BGN', 'Year Manifacture', 'Kilometers', 'Engine Type', 'Horsepower']]
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,2020,11 111 км,Бензинов двигател,Автоматична ск.кутия
1,Abarth 595,45800.0,2019,54 350 км,Бензинов двигател,181
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8 200 км,Бензинов двигател,165
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48 000 км,Бензинов двигател,160
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100 000 км,Бензинов двигател,278


In [24]:
df_auto['Kilometers'].unique()

array(['11 111 км', '54 350 км', '8 200 км', ..., '209 870 км',
       '177 850 км', '298 751 км'], dtype=object)

In [25]:
df_auto['Kilometers_test'] = df_auto['Kilometers'].str.extract('(\d+\ \d*)', expand=False)
df_auto['Kilometers_test'] = pd.to_numeric(df_auto['Kilometers_test'].str.replace(" ", ""),errors = 'coerce')
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Kilometers_test
0,Abarth 124,250000.0,2020,11 111 км,Бензинов двигател,Автоматична ск.кутия,11111.0
1,Abarth 595,45800.0,2019,54 350 км,Бензинов двигател,181,54350.0
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8 200 км,Бензинов двигател,165,8200.0
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48 000 км,Бензинов двигател,160,48000.0
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100 000 км,Бензинов двигател,278,100000.0


In [26]:
df_auto['Kilometers'] = df_auto['Kilometers_test']
del df_auto['Kilometers_test']

In [27]:
df_auto['Horsepower'].unique()

array(['Автоматична ск.кутия', '181', '165', '160', '278', '290',
       'Ръчна ск.кутия', 'Полуавтоматична ск.кутия', '400', '600', '177',
       '190', '239', '313', '245', '150', '272', '320', '122', '252',
       '143', '204', '435', '299', '354', '326', '233', '351', '230',
       '300', '258', '340', '180', '116', '140', '232', '350', '170',
       '286', '209', '371', '224', '450', '385', '265', '520', '211',
       '507', '560', '250', '610', '333', '200', '344', '420', '218',
       '110', '105', '510', '163', '310', '390', '240', '605', '124',
       '285', '131', '571', '11', '101', '335', '115', '155', '380',
       '102', '341', '275', '75', '223', '229', '125', '120', '90', '427',
       '255', '220', '408', '231', '130', '256', '270', '282', '225',
       '500', '95', '74', '360', '193', '353', '136', '365', '334', '570',
       '251', '106', '260', '517', '542', '707', '551', '725', '534',
       '550', '639', '132', '210', '175', '185', '135', '166', '78',
       '126'

In [28]:
df_auto['Horsepower'] = pd.to_numeric(df_auto['Horsepower'],errors = 'coerce')
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,2020,11111.0,Бензинов двигател,NaN
1,Abarth 595,45800.0,2019,54350.0,Бензинов двигател,181.0
2,Abarth 595 TURISMO*500*PANORAMA,51000.0,2021,8200.0,Бензинов двигател,165.0
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40900.0,2017,48000.0,Бензинов двигател,160.0
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",32500.0,2015,100000.0,Бензинов двигател,278.0


In [29]:
currentYear = datetime.now().year
df_auto['Years Automobile'] = currentYear - df_auto['Year Manifacture']

In [30]:
# rearrange column indexes
df_auto = df_auto[['Model', 'Year Manifacture','Years Automobile', 'Price BGN', 'Kilometers', 
                   'Engine Type', 'Horsepower']]
df_auto.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
0,Abarth 124,2020,2,250000.0,11111.0,Бензинов двигател,NaN
1,Abarth 595,2019,3,45800.0,54350.0,Бензинов двигател,181.0
2,Abarth 595 TURISMO*500*PANORAMA,2021,1,51000.0,8200.0,Бензинов двигател,165.0
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,2017,5,40900.0,48000.0,Бензинов двигател,160.0
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",2015,7,32500.0,100000.0,Бензинов двигател,278.0


# Data Filtering

In [31]:
df_auto =  df_auto.loc[(df_auto['Kilometers'] > 2000) ]
df_auto =  df_auto.loc[(df_auto['Years Automobile'] > 0)]              
df_auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23998 entries, 0 to 26069
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Model             23998 non-null  object 
 1   Year Manifacture  23998 non-null  int64  
 2   Years Automobile  23998 non-null  int64  
 3   Price BGN         23883 non-null  float64
 4   Kilometers        23998 non-null  float64
 5   Engine Type       23998 non-null  object 
 6   Horsepower        20918 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.5+ MB


In [32]:
df_auto["Year Manifacture"].unique()

array([2020, 2019, 2021, 2017, 2015, 1940, 1938, 2010, 2013, 2011, 2012,
       2014, 2016, 2008, 2018, 2009, 2005, 2002, 2006, 2007, 2004, 1999,
       2001, 2003, 2000, 1998, 1988, 1997, 1995, 1977, 1966, 1985, 1969,
       1958, 1996, 1990, 1994, 1983, 1967, 1991, 1984, 1980, 1979, 1974,
       1993, 1992, 1978, 1970, 1972, 1971, 1964, 1987, 1989, 1968, 1982,
       1986, 1959, 1975, 1962, 1976, 1960, 1973], dtype=int64)

In [33]:
df_auto.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
0,Abarth 124,2020,2,250000.0,11111.0,Бензинов двигател,NaN
1,Abarth 595,2019,3,45800.0,54350.0,Бензинов двигател,181.0
2,Abarth 595 TURISMO*500*PANORAMA,2021,1,51000.0,8200.0,Бензинов двигател,165.0
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,2017,5,40900.0,48000.0,Бензинов двигател,160.0
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",2015,7,32500.0,100000.0,Бензинов двигател,278.0


In [34]:
filter1 = df_auto['Engine Type'].isin(['Дизелов двигател'])
df_diesel = df_auto[filter1]
df_diesel

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
6,Acura Tsx,1940,82,400000.0,900000.0,Дизелов двигател,NaN
7,Acura Tsx,1940,82,400000.0,900000.0,Дизелов двигател,NaN
12,"Audi A5 2,0TDI177ksLEDEU5ВNAVISPORTBACKFACE5vrati",2013,9,26800.0,181000.0,Дизелов двигател,177.0
13,"Audi Q5 S-LINE*40 TDi-190ps 4x4 DSG*DISTRONIC,...",2019,3,69999.0,176000.0,Дизелов двигател,190.0
14,Audi Q5 S-LINE/F1/LED/NAVI/ TOP!!!СОБСТВЕН ЛИЗИНГ,2011,11,28000.0,190000.0,Дизелов двигател,239.0
...,...,...,...,...,...,...,...
26065,VW Amarok,2018,4,68200.0,85000.0,Дизелов двигател,258.0
26066,VW Touareg 3.0TDI,2014,8,30000.0,298751.0,Дизелов двигател,241.0
26067,VW Polo 1.6TDI EURO 5,2011,11,9300.0,191000.0,Дизелов двигател,90.0
26068,VW Touran *1.9 TDI* *101К.С. *ТОП* РЕГ. ОБСЛ.,2004,18,6700.0,219500.0,Дизелов двигател,101.0


In [35]:
filter2 = df_auto['Engine Type'].isin(['Бензинов двигател'])
df_gasoline = df_auto[filter2]
df_gasoline.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
0,Abarth 124,2020,2,250000.0,11111.0,Бензинов двигател,NaN
1,Abarth 595,2019,3,45800.0,54350.0,Бензинов двигател,181.0
2,Abarth 595 TURISMO*500*PANORAMA,2021,1,51000.0,8200.0,Бензинов двигател,165.0
3,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,2017,5,40900.0,48000.0,Бензинов двигател,160.0
4,"Acura Rdx 3.5V6 4X4,ПЕРФЕКТЕН!",2015,7,32500.0,100000.0,Бензинов двигател,278.0


In [36]:
filter3 = df_auto['Engine Type'].isin(['Електрически двигател'])
df_electric = df_auto[filter3]
df_electric.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
8,Acura Tsx,1938,84,400000.0,300000.0,Електрически двигател,NaN
36,Audi E-Tron Audi E-Tron Q4 50quattro sportback,2020,2,131700.0,13200.0,Електрически двигател,299.0
490,Audi Q4 35/ E-Tron/ MATRIX/ PANORAMA/ NAVI/ LA...,2021,1,109980.0,12500.0,Електрически двигател,170.0
493,Audi E-Tron SPORTBACK 55/ QUATTRO/ S-LINE/ 360...,2021,1,139980.0,23923.0,Електрически двигател,408.0
1821,Chevrolet Volt Десен волан,2012,10,19600.0,66000.0,Електрически двигател,151.0


In [37]:
filter4 = df_auto['Engine Type'].isin(['Хибриден двигател',
                                       'Plug-in хибрид двигател',
                                       'Plug-in хибрид'])
df_hybrid = df_auto[filter4]
df_hybrid.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
233,Audi Q7 3.0D E-TRON S-LINE,2016,6,79000.00,179000.0,Хибриден двигател,340.0
341,Audi Q5 2.0 TFSI Hybrid Quattro tiptronic S-LINE,2012,10,31500.00,147000.0,Хибриден двигател,211.0
362,Audi A6 HYBRID AUTOMATIC EURO 6D,2018,4,62500.00,223000.0,Хибриден двигател,204.0
1433,Bentley Bentayga HYBRID/ FIRST EDITION/ NAIM/ ...,2021,1,440661.00,27630.0,Хибриден двигател,449.0
1456,Bentley Bentayga FIRST EDITION,2021,1,448498.05,13000.0,Хибриден двигател,449.0


In [38]:
currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year
e ='%s.%s.%s'%(currentDay,currentMonth,currentYear)
file_name = 'car24.bg'
writer = pd.ExcelWriter(f'{file_name}{e}.xlsx', engine='openpyxl')
df_diesel.to_excel(writer, sheet_name='Diesel engines')
df_gasoline.to_excel(writer, sheet_name='Gasoline engines')
df_electric.to_excel(writer, sheet_name='Electric engines')
df_hybrid.to_excel(writer, sheet_name='Hybrid engines')
writer.save()